# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BUILDINGS_DISTANCES_PATH = f'{INPUT_PATH}/buildings_with_distances'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'BVNTN4SF3RHMH175',
  'HostId': 'TbKdRgJsq1+glKjTz1Vf8l/lnKjmoRWgML9qh0bHsaxMxwc36LQDTKvB6uv/npSUk0D4JQVt8Ac=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TbKdRgJsq1+glKjTz1Vf8l/lnKjmoRWgML9qh0bHsaxMxwc36LQDTKvB6uv/npSUk0D4JQVt8Ac=',
   'x-amz-request-id': 'BVNTN4SF3RHMH175',
   'date': 'Wed, 26 Mar 2025 23:00:58 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=8,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-26 18:01:00,521][INFO    ][coiled] Fetching latest package priorities...
[2025-03-26 18:01:00,523][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-03-26 18:01:01,396][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-03-26 18:01:01,405][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-03-26 18:01:02,391][INFO    ][coiled] Running pip check...
[2025-03-26 18:01:04,019][INFO    ][coiled] Validating environment...
[2025-03-26 18:01:06,186][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-03-26 18:01:06,486][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-03-26 18:01:06,488][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-03-26 18:01:07,438][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-ntylm.dask.host/atafD5pdumjdrwos/status


Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/iostream.py", line 1367, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/ssl.py", line 1319, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 202, in _handle_events
    handler_func(fileobj, events)
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/pyth

# RUN

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [7]:
import dask_geopandas as dgpd
import geopandas as gpd
from shapely.geometry import shape, Point, LineString, Polygon, MultiPolygon, MultiLineString
from shapely.geometry.base import BaseGeometry
from shapely.strtree import STRtree
from shapely.errors import ShapelyError
import dask
from dask import delayed
import pandas as pd
import numpy as np
from shapely.wkb import loads as wkb_loads
from dask import compute
from citywide_calculation import get_utm_crs

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

def load_dataset(path, epsg=None):
    dataset = dgpd.read_parquet(path, npartitions=4)
    
    # Only assign if the file has no CRS
    if epsg:
        if dataset.crs is None:
            dataset = dataset.set_crs("EPSG:4326")  # assume WGS84 if missing
        dataset = dataset.to_crs(epsg)

    return dataset


max_distance = 200
default_distance = 500

def compute_distance_partition(buildings_df, roads_geom_list, max_distance, default_distance):
    tree = STRtree(roads_geom_list)

    def distance_fn(bgeom):
        try:
            bgeom = shape(bgeom) if not isinstance(bgeom, BaseGeometry) else bgeom
            nearby_indices = tree.query(bgeom.buffer(max_distance))
            if nearby_indices is None or len(nearby_indices) == 0:
                return default_distance
            nearby_geoms = [roads_geom_list[i] for i in nearby_indices]
            return min(bgeom.distance(road) for road in nearby_geoms)
        except Exception:
            return default_distance

    buildings_df = buildings_df.copy()
    buildings_df['geometry'] = buildings_df['geometry'].apply(shape)  # extra safe
    buildings_df["distance_to_nearest_road"] = buildings_df.geometry.apply(distance_fn)
    return buildings_df


@delayed
def calculate_building_distances_to_roads(city_name):
    paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet',
    'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet'
    }
    epsg = get_epsg(city_name).compute()  
    # Load and prepare roads for spatial index
    roads = load_dataset(paths['roads'], epsg=epsg).compute()
    roads_geom_list = [geom for geom in roads.geometry]

    # Load buildings lazily
    buildings = load_dataset(paths['buildings'], epsg=epsg)

    meta = buildings._meta.assign(distance_to_nearest_road='f8')

    # Apply distance computation per partition
    buildings_with_dist = buildings.map_partitions(
        compute_distance_partition,
        roads_geom_list,
        max_distance,
        default_distance,
        meta=meta
    )

    # Write output
    columns_to_keep = ['id', 'geometry','distance_to_nearest_road']
    buildings_with_dist = buildings_with_dist[columns_to_keep].set_index('id')
    out_path = paths['buildings'].replace(".geoparquet", "_with_distances.geoparquet")
    buildings_with_dist.to_parquet(out_path)
    return out_path


import time

start = time.time()

city_list = ["Nairobi","Belo_Horizonte"]

def run_all_cities(city_list):
    delayed_jobs = [delayed(calculate_building_distances_to_roads)(city) for city in city_list]
    results = compute(*delayed_jobs)
    return results

run_all_cities(city_list)

print("Elapsed:", time.time() - start)


Elapsed: 1650.1406290531158


In [8]:
city_name = 'Nairobi'
nai_buildings = dgpd.read_parquet(f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet', npartitions=4)

In [ ]:
from scipy.stats import t, sem, entropy

def calculate_azimuth(buildings_partition, bin_width_degrees):
        
    buildings_partition.loc[:, 'azimuth'] = buildings_partition['geometry'].apply(lambda x: calculate_azimuth(buildings_partition(x))  % 90. )

    # Extract azimuths (assuming they are in degrees)
    azimuths = buildings_partition['azimuth'].values

    # Define the number of bins (e.g., 0-360 degrees, based on bin width)
    num_bins = int(90 / bin_width_degrees)

    # Create a histogram of azimuths (observed distribution P)
    histogram, bin_edges = np.histogram(azimuths, bins=num_bins, range=(0, 90))
    
    # Normalize the histogram to make it a probability distribution P
    P = histogram / histogram.sum()

    # Create a uniform distribution Q with the same number of bins
    Q = np.ones(num_bins) / num_bins

    # Calculate the KL divergence
    kl_divergence = entropy(P, Q)

    # Standardize the KL divergence by dividing by log(n)
    max_kl_divergence = np.log(num_bins)
    standardized_kl_divergence = kl_divergence / max_kl_divergence


def building_distance_metrics(city_name):

    grid_cell_count = 0
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    # Get EPSG
    epsg = get_epsg(city_name).compute()
    # Load grid
    grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    
    buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)#.compute()
    buildings['distance_to_nearest_road'] = buildings['distance_to_nearest_road'].astype(float)
    buildings['area'] = buildings.geometry.area
    joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
    counts_buildings = joined_buildings.groupby('index_right').size()
    grid['n_buildings'] = grid.index.map(counts_buildings).fillna(0).astype(int)
    average_distance = joined_buildings.groupby('index_right')['distance_to_nearest_road'].mean()
    grid['average_distance_nearest_building'] = grid.index.map(average_distance).fillna(0).astype(float)
 

    buildings_closer_than_20m = buildings[buildings['distance_to_nearest_road'] <= 20]
    joined_buildings_closer_than_20m = dgpd.sjoin(buildings_closer_than_20m, grid, predicate='intersects') 
    n_buildings_closer_than_20m = joined_buildings_closer_than_20m.groupby('index_right').size()
    grid['n_buildings_closer_than_20m'] = grid.index.map(n_buildings_closer_than_20m).fillna(0).astype(float)

    grid['m1'] = grid['n_buildings_closer_than_20m'] / grid['n_buildings']
    grid['m2'] = grid['average_distance_nearest_building']

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_1_2.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)

import time

start = time.time()

city_list = ["Nairobi","Belo_Horizonte"]

def run_all_cities_metrics1_2(city_list):
    delayed_jobs = [delayed(building_distance_metrics)(city) for city in city_list]
    results = compute(*delayed_jobs)
    return results

run_all_cities_metrics1_2(city_list)

print("Elapsed:", time.time() - start)

Elapsed: 88.6606342792511


In [15]:
nai_buildings.distance_to_nearest_road = nai_buildings.distance_to_nearest_road.astype(float)

In [17]:
nai_buildings.to_file('nairobi_buildings_with_distance.shp')

/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_33008/1568468617.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nai_buildings.to_file('nairobi_buildings_with_distance.shp')
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'distance_to_nearest_road' to 'distance_t'
  ogr_write(


KeyboardInterrupt: 

In [7]:
client.close()